In [5]:
import tensorflow as tf
import numpy as np

In [13]:
class TwoLayerLogicGate(tf.Module):
    def __init__(self):
        super().__init__()
        self.built = False

    def __call__(self, x, train=True):
        if not self.built:
            self.w1 = tf.Variable(tf.random.normal([2, 2]), name="weights1")
            self.b1 = tf.Variable(tf.zeros([2]), name="bias1")
            self.w2 = tf.Variable(tf.random.normal([2, 1]), name="weights2")
            self.b2 = tf.Variable(tf.zeros([1]), name="bias2")
            self.built = True

        hidden = tf.sigmoid(tf.matmul(x, self.w1) + self.b1)
        output = tf.sigmoid(tf.matmul(hidden, self.w2) + self.b2)
        return output

def compute_loss(y_pred, y_true):
    return tf.reduce_mean(tf.square(y_pred - y_true))

def train_model(model, x_train, y_train, learning_rate=0.5, epochs=5000):
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            y_pred = model(x_train)
            loss = compute_loss(y_pred, y_train)

        grads = tape.gradient(loss, model.variables)
        for g, v in zip(grads, model.variables):
            v.assign_sub(learning_rate * g)

        if epoch % 1000 == 0:
            acc = compute_accuracy(model, x_train, y_train)
            print(f"Epoch {epoch}, Loss: {loss.numpy():.4f}, Accuracy: {acc:.4f}")

def compute_accuracy(model, x, y_true):
    y_pred = model(x, train=False)
    y_pred_rounded = tf.round(y_pred)
    correct = tf.equal(y_pred_rounded, y_true)
    return tf.reduce_mean(tf.cast(correct, tf.float32)).numpy()



In [ ]:
xor_table = np.array([[0, 0, 0],
                      [1, 0, 1],
                      [0, 1, 1],
                      [1, 1, 0]], dtype=np.float32)

x_train = xor_table[:, :2]
y_train = xor_table[:, 2:]

model = TwoLayerLogicGate()
train_model(model, x_train, y_train)

w1 = model.w1.numpy()
w2 = model.w2.numpy()
b1 = model.b1.numpy()
b2 = model.b2.numpy()[0]
print(f"\nLearned weights for first layer:\n{w1}")
print(f"Learned weights for second layer:\n{w2}")
print(f"Learned bias for first layer: {b1}")
print(f"Learned bias for second layer: {b2}\n")

y_pred = model(x_train, train=False).numpy().round().astype(np.uint8)
print("Predicted Truth Table:")
print(np.column_stack((xor_table[:, :2], y_pred)))